In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HTML
from tvbwidgets.ui.connectivity_matrix_editor_widget import ConnectivityMatrixEditor
from tvbwidgets.ui.spacetime_widget import SpaceTimeVisualizerWidget
from tvb.datatypes.connectivity import Connectivity
import numpy as np

# import connectivity
conn = Connectivity.from_file()
conn.configure()


# create custom widget
class CustomSpaceTimeEditor(SpaceTimeVisualizerWidget, widgets.VBox):
    def __init__(self, connectivity, width=600, height=600):
        SpaceTimeVisualizerWidget.__init__(self, connectivity, width=width, height=height)#Initialize widget
        widgets.VBox.__init__(self)#Initialize ipywidget
        
        self.conduction_speed = self.options.children[0].value
        
        
        self.update_plot_details = HTML(
            value=self.update_generate_details(connectivity.tract_lengths.max()),
            layout=self.DEFAULT_BORDER
        )

        # Only use a single plot output and details section
        self.children = [widgets.Output(), self.update_plot_details]
        with self.children[0]:
            self.display()

    def on_update_max_tract_length(self, new_max_tract_length):#function that is called when max_tract_length updates
        self.graphs_matplotlib.clear_output(wait=True)

        self.conduction_speed = self.options.children[0].value

        self.options.children[1].max = new_max_tract_length / self.conduction_speed
        self.options.children[2].max = new_max_tract_length / self.conduction_speed

        self.options.children[1].min = self.connectivity.tract_lengths.min() / self.conduction_speed
        self.options.children[2].min = self.connectivity.tract_lengths.min() / self.conduction_speed

        self.options.children[1].value = self.options.children[1].min
        self.options.children[2].value = self.options.children[2].max

        self.from_time = self.options.children[1].value
        self.to_time = self.options.children[2].value

        self.refresh_plot_details(new_max_tract_length)

        # Update slices
        for i in range(len(self.graph_slices)):
            connectivity = self._prepare_connectivity(i)
            data = self._generate_colors(connectivity)
            self.graph_slices[i].material.map.data = np.fliplr(data)
            self.ims[i].imshow(data)

        with self.graphs_matplotlib:
            display(self.fig)

    def refresh_plot_details(self, new_max_tract_length):
        self.update_plot_details.value = self.update_generate_details(new_max_tract_length)

    def update_generate_details(self, new_max_tract_length):
        return f"""<br>
            <div style="line-height:1px;">
            <br><h3> UPDATED PLOT DETAILS</h3> <br><br>
            <h4>conduction speed:</h4><br> 
            {self.conduction_speed} mm/ms<br>
            <h4>min(non-zero)</h4><h4>delay:</h4><br>
            {np.min(self.connectivity.tract_lengths[np.nonzero(self.connectivity.tract_lengths)]) / self.conduction_speed:.2f} ms<br>
            <h4>max delay:</h4><br>
            {new_max_tract_length / self.conduction_speed:.2f} ms<br>
            <h4>min(non-zero)</h4><h4>tract length:</h4><br>
            {np.min(self.connectivity.tract_lengths[np.nonzero(self.connectivity.tract_lengths)])} mm<br>
            <h4>max tract length:</h4><br>
            {new_max_tract_length} mm<br>
            <h4>min(non-zero)</h4><h4>weight:</h4><br>
            {np.min(self.connectivity.weights[np.nonzero(self.connectivity.weights)])}<br>
            <h4>max weight:</h4><br>
            {np.max(self.connectivity.weights)}<br><br></div>"""



class CustomMatrixEditor(widgets.VBox, ConnectivityMatrixEditor):
    def __init__(self, connectivity, viewer, **kwargs):
        ConnectivityMatrixEditor.__init__(self, connectivity, **kwargs)
        widgets.VBox.__init__(self)
        self.viewer = viewer

        self.children = [widgets.Output()]
        with self.children[0]:
            self.display()

    def on_apply_change(self, change):
        super().on_apply_change(change)
        if hasattr(self, "new_max_tract_length") and self.new_max_tract_length is not None:
            self.viewer.on_update_max_tract_length(self.new_max_tract_length)


# Initialzie widgets
viewer = CustomSpaceTimeEditor(connectivity=conn)
editor = CustomMatrixEditor(connectivity=conn, viewer=viewer)

# Display widgets side by side
hbox = widgets.HBox([editor, viewer], 
                   layout=widgets.Layout(
                       width='100%',
                       justify_content='space-between',
                       align_items='flex-start'
                   ))

display(hbox)

2025-04-07 14:58:09,279 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.
